<a href="https://colab.research.google.com/github/MiguelOrtega630/IA_2025/blob/main/Taller_Creditos_7_Noviembre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.impute import SimpleImputer



print('\nCargando /content/creditos.csv...')
try:
    df = pd.read_csv('/content/creditos.csv')
except FileNotFoundError:
    raise FileNotFoundError("No se encontró '/content/creditos.csv'. Sube el archivo en Colab antes de ejecutar.")


print('\nDimensiones del dataset:', df.shape)
print('\nColumnas:')
print(df.columns.tolist())
print('\nPrimeras filas:')
print(df.head())



common_names = ['target','class','label','y','default','Default','DEFAULT','aprobado','Aprobado','aprobación','aprobacion','STATUS','status','Status','approved']
target_col = None
for name in common_names:
    if name in df.columns:
        target_col = name
        break
if target_col is None:
    target_col = df.columns[-1]
    print(f"\nNo se detectó una etiqueta común. Usaré la última columna como target: '{target_col}'")
else:
    print(f"\nColumna objetivo detectada: '{target_col}'")



y = df[target_col]
if y.dtype == 'O' or y.dtype.name == 'category' or y.dtype == bool:
    y = pd.factorize(y)[0]
    print('\nTarget factorized (categórico -> numérico).')
else:
    y = y.values



X = df.drop(columns=[target_col]).copy()



numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include=['object','category','bool']).columns.tolist()
print('\nColumnas numéricas:', numeric_cols)
print('Columnas categóricas:', cat_cols)



numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor_default = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, cat_cols)
])



numeric_for_hamming = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('kbins', KBinsDiscretizer(n_bins=4, encode='onehot-dense', strategy='quantile'))
])

print('\nFin del ejercicio.')


Cargando /content/creditos.csv...

Dimensiones del dataset: (200, 3)

Columnas:
['edad', 'credito', 'cumplio']

Primeras filas:
   edad  credito  cumplio
0    18   363112        0
1    19   477965        1
2    20   239072        0
3    22   195265        0
4    22   482174        0

No se detectó una etiqueta común. Usaré la última columna como target: 'cumplio'

Columnas numéricas: ['edad', 'credito']
Columnas categóricas: []

Fin del ejercicio.


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [13]:
distancias = ['euclidean', 'manhattan', 'minkowski', 'hamming']

for dist in distancias:
    print(f'\n=== Distancia: {dist.upper()} ===')


    if dist == 'hamming':
        preprocessor = ColumnTransformer(transformers=[
            ('num', numeric_for_hamming, numeric_cols),
            ('cat', cat_transformer, cat_cols)
        ])
    else:
        preprocessor = preprocessor_default


    modelo = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('knn', KNeighborsClassifier(n_neighbors=5, metric=dist))
    ])

    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    print('\nMatriz de confusión:')
    print(confusion_matrix(y_test, y_pred))
    print('\nReporte de clasificación:')
    print(classification_report(y_test, y_pred))
    print('Accuracy:', accuracy_score(y_test, y_pred))


=== Distancia: EUCLIDEAN ===

Matriz de confusión:
[[ 2  8]
 [ 3 47]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.40      0.20      0.27        10
           1       0.85      0.94      0.90        50

    accuracy                           0.82        60
   macro avg       0.63      0.57      0.58        60
weighted avg       0.78      0.82      0.79        60

Accuracy: 0.8166666666666667

=== Distancia: MANHATTAN ===

Matriz de confusión:
[[ 2  8]
 [ 3 47]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.40      0.20      0.27        10
           1       0.85      0.94      0.90        50

    accuracy                           0.82        60
   macro avg       0.63      0.57      0.58        60
weighted avg       0.78      0.82      0.79        60

Accuracy: 0.8166666666666667

=== Distancia: MINKOWSKI ===

Matriz de confusión:
[[ 2  8]
 [ 3 47]]

Reporte de clasifica

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
mejor_distancia = 'manhattan'
tam = len(df)
valores_k = [3, 5, 7, 9, int(np.sqrt(tam))]

print(f"\nUsando la mejor distancia: {mejor_distancia}")
for k in valores_k:
    print(f'\n--- k = {k} ---')
    modelo = Pipeline(steps=[
        ('preprocessor', preprocessor_default),
        ('knn', KNeighborsClassifier(n_neighbors=k, metric=mejor_distancia))
    ])
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, y_pred))


Usando la mejor distancia: manhattan

--- k = 3 ---
Accuracy: 0.8166666666666667

--- k = 5 ---
Accuracy: 0.8166666666666667

--- k = 7 ---
Accuracy: 0.8166666666666667

--- k = 9 ---
Accuracy: 0.8166666666666667

--- k = 14 ---
Accuracy: 0.8333333333333334


Al aumentar k, el modelo tiende a ser más suave (menos sobreajuste), pero puede perder precisión si k es demasiado grande.
En cambio, valores pequeños de k pueden capturar mejor los patrones locales, pero con más riesgo de overfitting.